In [7]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
import pydicom
from PIL import Image

In [8]:
def load_dicom_image(path, img_size=256):
    # Load dicom and convert to 256x256 RGB image
    dicom = pydicom.dcmread(path)
    image = dicom.pixel_array
    image = Image.fromarray(image).convert('RGB')
    image = image.resize((img_size, img_size))
    image = np.array(image) / 255.0

    # Convert to TensorFlow tensor
    print(f"Loaded image shape: {image.shape}, dtype: {image.dtype}")
    return tf.convert_to_tensor(image, dtype=tf.float32)

In [9]:
def process_directory(directory_path, label, img_size=256):
    images, labels = [], []
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            # Check if the file is a DICOM file
            if filename.endswith('.dcm'):
                # Construct the full path to the DICOM file
                file_path = os.path.join(root, filename)
                # Load and process the image
                image = load_dicom_image(file_path, img_size=img_size)
                # Append the processed image and its label to the lists
                images.append(image)
                labels.append(label)
    return images, labels

In [10]:
def create_datasets(cancer_dir, non_cancer_dir, img_size=256):
    # Load and process images and labels
    cancer_images, cancer_labels = process_directory(cancer_dir, 1, img_size=img_size)
    non_cancer_images, non_cancer_labels = process_directory(non_cancer_dir, 0, img_size=img_size)
    
    # Combine images and labels
    all_images = np.concatenate([np.array(cancer_images), np.array(non_cancer_images)], axis=0)
    all_labels = np.array(cancer_labels + non_cancer_labels)

    # After combining images and labels
    print(f"all_images shape: {all_images.shape}, dtype: {all_images.dtype}")
    print(f"all_labels shape: {all_labels.shape}, dtype: {all_labels.dtype}")

    
    return all_images, all_labels

In [11]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

def build_custom_model(input_shape=(256, 256, 3), dropout_rate=0.5, l2_reg=0.001):
    inputs = Input(shape=input_shape)
    
    # Example of a simple CNN architecture
    x = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg))(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Flatten()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [12]:
# Load all data
all_images, all_labels = create_datasets(
    cancer_dir='/Users/arjunmoorthy/Desktop/Research_Capstone/ImageData/final_train_data/cancer',
    non_cancer_dir='/Users/arjunmoorthy/Desktop/Research_Capstone/ImageData/final_train_data/non_cancer',
    img_size=256)

# Split data into training+validation and test sets
(train_val_images, test_images, train_val_labels, test_labels) = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42)

# Prepare for 5-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64


In [13]:
from tensorflow.keras.metrics import Precision, Recall, AUC

# Assuming kf, train_val_images, and train_val_labels are already defined
for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_images)):
    print(f"Training dataset shape: {train_val_images[train_idx].shape}")
    print(f"Training labels shape: {train_val_labels[train_idx].shape}")
    print(f"Training on fold {fold+1}/5...")
    
    # Generate datasets for the current fold
    train_dataset = tf.data.Dataset.from_tensor_slices((train_val_images[train_idx], train_val_labels[train_idx]))
    train_dataset = train_dataset.shuffle(buffer_size=len(train_idx)).batch(32).prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_tensor_slices((train_val_images[val_idx], train_val_labels[val_idx]))
    val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    
    # Build and compile the custom model instead of the ResNet-based model
    model = build_custom_model(input_shape=(256, 256, 3), dropout_rate=0.5, l2_reg=0.001)
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])
    
    # Fit model
    history = model.fit(train_dataset, epochs=50, validation_data=val_dataset)


Training dataset shape: (29907, 256, 256, 3)
Training labels shape: (29907,)
Training on fold 1/5...
Epoch 1/50


2024-04-15 19:06:38.315449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


935/935 ━━━━━━━━━━━━━━━━━━━━ 124s 123ms/step - accuracy: 0.5893 - auc: 0.4945 - loss: 0.7088 - precision: 0.5909 - recall: 0.9912 - val_accuracy: 0.5873 - val_auc: 0.5000 - val_loss: 0.6781 - val_precision: 0.5873 - val_recall: 1.0000
Epoch 2/50
935/935 ━━━━━━━━━━━━━━━━━━━━ 116s 124ms/step - accuracy: 0.5883 - auc: 0.4945 - loss: 0.6777 - precision: 0.5883 - recall: 1.0000 - val_accuracy: 0.5873 - val_auc: 0.5000 - val_loss: 0.6778 - val_precision: 0.5873 - val_recall: 1.0000
Epoch 3/50
935/935 ━━━━━━━━━━━━━━━━━━━━ 112s 120ms/step - accuracy: 0.5876 - auc: 0.4942 - loss: 0.6777 - precision: 0.5876 - recall: 1.0000 - val_accuracy: 0.5873 - val_auc: 0.5000 - val_loss: 0.6778 - val_precision: 0.5873 - val_recall: 1.0000
Epoch 4/50
935/935 ━━━━━━━━━━━━━━━━━━━━ 114s 122ms/step - accuracy: 0.5883 - auc: 0.4949 - loss: 0.6775 - precision: 0.5883 - recall: 1.0000 - val_accuracy: 0.5873 - val_auc: 0.5000 - val_loss: 0.6778 - val_precision: 0.5873 - val_recall: 1.0000
Epoch 5/50
935/935 ━━━━━━━━

: 

In [ ]:
# After cross-validation, you might want to retrain your model on all training data and evaluate it on the test set
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)
# Make sure to re-build and compile your model

# Assuming 'model' is your final trained model
test_loss, test_acc, test_precision, test_recall, test_auc = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")